# CHARISMA: Fourier transform Raman peak interpolation

after Hutsebaut, Didier, Peter Vandenabeele, and Luc Moens. "Evaluation of an accurate calibration and spectral standardization procedure for Raman spectroscopy." Analyst 130.8 (2005): 1204-1214.

James Thomson & Bastian Barton, May 18, 2021

In [ ]:
## Pixel start and stop positions, for x-axis
freq_start = 845
freq_stop = 856

In [ ]:
import numpy as np
## Pixel series for x-axis
x = np.linspace(freq_start,freq_stop,(freq_stop-freq_start+1))

In [ ]:
## Intensity series for Peak
y = np.array([11065.8,14503.13,18909.2,27798.07,46242.13,62690.6,50688.33,27785.47,14060.27,8208.467,5984.533,5286.533])

In [ ]:
# Normalize
y -= np.min(y)
y /= np.max(y)

In [ ]:
# x and y have even lengths, which is bad for FFT centering. Truncate last value:
y = y[:-1]
x = x[:-1]

In [ ]:
import matplotlib.pyplot  as plt
%matplotlib inline

In [ ]:
plt.figure()
plt.plot(x, y)

Fourier transform, extended 64-fold by zero filling

In [ ]:
## Tranform intensities into fourier domain
y_f = np.fft.fft(y)

In [ ]:
# Check FFT modulus
plt.figure()
plt.plot(y_f)

In [ ]:
# Information is contained in left half
plt.figure()
plt.plot(y_f[0:7])

In [ ]:
len(y_f)

In [ ]:
len(y_f)//2+2

Pad middle (large periodicities) with zeros

In [ ]:
zeropad = np.zeros(2000)
ext_y_f = np.hstack((y_f[0:7], zeropad, y_f[7:]))
ext_y_f.shape

In [ ]:
plt.figure()
plt.plot(ext_y_f)

In [ ]:
## Inverse FFT
y_if = np.real(np.fft.ifft(ext_y_f))
y_if -= np.min(y_if)
y_if /= np.max(y_if)

In [ ]:
## Create new x-axis
ext_x = np.linspace(freq_start,freq_stop,len(y_if))

In [ ]:
plt.figure()
plt.plot(ext_x, y_if)
plt.plot(x,y)

Cut out peak tip & fit parabola

In [ ]:
d = 200
x1 = ext_x[np.argmax(y_if)-d:np.argmax(y_if)+d]
y1 = y_if[np.argmax(y_if)-d:np.argmax(y_if)+d]

In [ ]:
z = np.polyfit(x1,y1,2)

In [ ]:
p = np.poly1d(z)

In [ ]:
plt.figure()
plt.plot(x1,y1)
plt.plot(x1, p(x1))

Looks nice for this here data. Maximum of fitted peak at:

In [ ]:
x1[np.argmax(p(x1))]